In [1]:
import os
import sys
import multiprocessing
import warnings

import pandas as pd
import numpy as np

from Bio import SeqIO
from glob import glob
from functools import partial
sys.path.append('../../')
from hamp_pred.external.lbs.sequence import mmseqs2

In [2]:
data_dir = '../../data/input'

# Get HAMP sequences

In [3]:
# read and cluster sequences from https://pubmed.ncbi.nlm.nih.gov/20184894/
msa = list(SeqIO.parse(os.path.join(data_dir, 'hamp_msa.fasta'), 'fasta'))

msa_df = pd.DataFrame(
        [(str(i.seq).replace('-', ''), i.id) for i in msa],
    columns=['sequence', 'id']
)

In [4]:
len(msa)

6456

In [5]:
# remove redundancy at 70% ident and 70% coverage
cl = mmseqs2.MMSeqsClusterer()
msa_df_clustered = cl.cluster(msa_df, min_identity=0.7, coverage=0.7)

createdb /tmp/v7q8l3k3 tmp/1611238168804029899/input --max-seq-len 65535 --dont-split-seq-by-len 1 --dbtype 0 --dont-shuffle 1 --id-offset 0 --compressed 0 -v 3 

Converting sequences
[
Time for merging files: 0h 0m 0s 12ms
Time for merging files: 0h 0m 0s 6ms
Time for merging files: 0h 0m 0s 0ms
Time for processing: 0h 0m 0s 46ms
kmermatcher tmp/1611238168804029899/input tmp/1611238168804029899/clu_tmp/2538666772622062916/linclust/961887237917144505/pref --sub-mat blosum62.out --alph-size 13 --min-seq-id 0.7 --kmer-per-seq 21 --adjust-kmer-len 0 --mask 0 --mask-lower-case 0 --cov-mode 0 -k 0 -c 0.7 --max-seq-len 65535 --hash-shift 5 --split-memory-limit 0 --include-only-extendable 0 --skip-n-repeat-kmer 0 --threads 20 --compressed 0 -v 3 

Database size: 6456 type: Aminoacid
Reduced amino acid alphabet: (A S T) (C) (D B N) (E Q Z) (F Y) (G) (H) (I V) (K R) (L J M) (P) (W) (X) 

Estimated memory consumption 2 MB
Generate k-mers list for 1 split
[========================================

[=================================================================] 5.56K 0s 4ms
Index statistics
Entries:          83252
DB size:          488 MB
Avg k-mer size:   0.001301
Top 10 k-mers
    DEGLTF	82
    EIQAFN	62
    DEGLNF	49
    EIDAFN	35
    RREGAR	32
    EIQSFN	21
    PVSDGQ	20
    IGMREV	19
    DEGMTV	16
    PVSDGR	14
Time for index table init: 0h 0m 0s 541ms
k-mer similarity threshold: 154
	k-mers per position = 0.475694, k-mer match probability: 0.000000
k-mer match probability: 0.000000

Starting prefiltering scores calculation (step 1 of 1)
Query db start  1 to 5561
Target db start  1 to 5561
[================================================================] 5.56K 0s 4ms
=
0.473296 k-mers per position
27 DB matches per sequence
0 overflows
6 sequences passed prefiltering per query sequence
3 median result list length
0 sequences with 0 size result lists

Time for prefiltering scores calculation: 0h 0m 0s 10ms
Time for merging files: 0h 0m 0s 4ms
Time for processing: 0h 0m 0

In [6]:
msa_df_clustered = msa_df_clustered.groupby('clust_id').head(1)
len(msa_df_clustered)

5388

In [7]:
msa_df_clustered.to_pickle(os.path.join(data_dir, 'hamp_master.p'))

# Unpack AF2 models

In [21]:
def extract_models(models_dir, out_path, df_clustered):
    for out_file in glob(os.path.join(models_dir, '*', '*result.zip')):

        #print(out_file)
        
        seq_id = int(out_file.split('/')[-1].replace('.result.zip', '').split("_")[1])
        assert seq_id in df_clustered.index

        tmp = os.path.join(out_path, str(seq_id))
        
        if not os.path.exists(tmp):
        
            os.system(f'mkdir -p {tmp}')

            # -n ensures that existing files are not overwritten
            os.system(f'unzip -n -j "{out_file}" "*relaxed*" -d {tmp}')
        
        #break

In [22]:
#
# af2 model files are avaliable upon request 
#
models_dir = '/home/nfs/jludwiczak/calc/hamp_olek/hamp_final/out'
out_path = os.path.join(data_dir, 'af2_full')
df_clustered = pd.read_pickle(os.path.join(data_dir, 'hamp_master.p'))

# get aligned sequences
msa = list(SeqIO.parse(os.path.join(data_dir, 'hamp_msa.fasta'), 'fasta'))
hampid2alnseq = pd.DataFrame(
        [(str(i.seq), i.id) for i in msa],
    columns=['sequence', 'id']
)

In [23]:
len(df_clustered), len(msa), len(hampid2alnseq)

(5388, 6456, 6456)

In [24]:
# unzip af2 models bundles
# uncomment me
extract_models(models_dir, out_path, df_clustered)

Archive:  /home/nfs/jludwiczak/calc/hamp_olek/hamp_final/out/hamp_52/hamp_6441.result.zip
 extracting: ../../data/input/af2_full/6441/hamp_6441_unrelaxed_rank_1_model_2.pdb  
 extracting: ../../data/input/af2_full/6441/hamp_6441_unrelaxed_rank_1_model_2_scores.json  
 extracting: ../../data/input/af2_full/6441/hamp_6441_relaxed_rank_1_model_2.pdb  
 extracting: ../../data/input/af2_full/6441/hamp_6441_unrelaxed_rank_2_model_1.pdb  
 extracting: ../../data/input/af2_full/6441/hamp_6441_unrelaxed_rank_2_model_1_scores.json  
 extracting: ../../data/input/af2_full/6441/hamp_6441_relaxed_rank_2_model_1.pdb  
 extracting: ../../data/input/af2_full/6441/hamp_6441_unrelaxed_rank_3_model_3.pdb  
 extracting: ../../data/input/af2_full/6441/hamp_6441_unrelaxed_rank_3_model_3_scores.json  
 extracting: ../../data/input/af2_full/6441/hamp_6441_relaxed_rank_3_model_3.pdb  
 extracting: ../../data/input/af2_full/6441/hamp_6441_unrelaxed_rank_4_model_5.pdb  
 extracting: ../../data/input/af2_full/644

# Analyses AF2 models


In [25]:
### Measure with SamCC
### Get AF2 scores
### Store PDB file link

In [26]:
# To measure af2 hamp models use samCC turbo https://academic.oup.com/bioinformatics/article/36/22-23/5368/6039120
sys.path.append('../../hamp_pred/')
from utils.measure import measure_one_HAMP, get_ref_crick
from utils.tools import diffangle

In [27]:
def run_multiprocess(func, tasks, n_cores, tasks_per_core=1):  
    stdout_queue = multiprocessing.Queue()
    pool = multiprocessing.Pool(processes=n_cores, initargs=[stdout_queue], maxtasksperchild=tasks_per_core)
    for i, data in enumerate(pool.map(func, tasks), 1):
        yield data
    pool.close()
    pool.join()

In [28]:
# referece Crick angles
crangles = {'a':19.5,'b':122.35,'c':-134.78,'d': -31.92,'e':70.92 ,'f':173.78,'g':-83.35}

# aa names mapping
AA_3_to_1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
             'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
             'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
             'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

In [29]:
# define ranges for helix 1 and helix 2 in MSA
h1_msa_start = 5 #4
h1_msa_stop = 18

h2_msa_start = 85 # 84
h2_msa_stop = 98

start_hep = 'a' #g

In [30]:
warnings.filterwarnings(action='ignore', category=UserWarning)
import json

In [31]:
data=[]

for idx in df_clustered.index:
    
    row = df_clustered.loc[idx]
    group = row['id'].split("|")[1]
    true_id = int(row['id'].split("|")[0])
        
    # aligned sequence
    alnseq = hampid2alnseq.loc[idx].sequence
    
    # cut helix1 and helix2
    h1 = alnseq[h1_msa_start:h1_msa_stop].replace('-', '')
    h2 = alnseq[h2_msa_start:h2_msa_stop].replace('-', '')
    
    if len(h1) != len(h2): 
        continue
        
    seq = row.sequence    
    if seq.find('X')>-1: 
        continue

    # measure model        
    h1_start = seq.find(h1); assert h1_start > -1
    h2_start = seq.find(h2); assert h2_start > -1
    
    a1_start, a1_stop = h1_start+1, h1_start+len(h1)+1
    a2_start, a2_stop = h2_start+1, h2_start+len(h2)+1
    chain1, chain2 = 'A', 'B'
    
    kwargs = {'a1_start':a1_start, 
              'a1_stop':a1_stop,
              'a2_start':a2_start,
              'a2_stop':a2_stop,
              'chain1':chain1,
              'chain2':chain2}
    
    
    
    mapfunc = partial(measure_one_HAMP, **kwargs)
    
    # get 1 rank af2 model
    pdb_files = sorted(glob(os.path.join(out_path, str(idx), '*_relaxed*.pdb')), key=lambda x:int(x.split('/')[-1].split("_")[4]))
    
    # analyse only rank 1 model
    pdb_files = [pdb_files[0]]
    
    # analyse selected models 
    for job, pdb_file in zip(run_multiprocess(mapfunc, pdb_files, len(pdb_files)), pdb_files):    
                
        dir_path, filename = os.path.split(pdb_file)
        _, last_dir = os.path.split(dir_path)
        assert idx == int(last_dir)
        pdb_file_simple = os.path.join(last_dir, filename)
        
        
        # parse scores 'max_pae', 'pae', 'plddt', 'ptm'
        json_file = pdb_file[:-4].replace('relaxed', 'unrelaxed') + "_scores.json"
        scores = json.load(open(json_file))
        
        # measure with SamCC
        
        bundle_df, n_crick_mut, c_crick_mut = job

        n_crick_mut = n_crick_mut[2:-2]
        c_crick_mut = c_crick_mut[2:-2]
        n_P = c_P = 3.5 

        c_phi = n_phi = crangles[start_hep] 

        n_crick_ref = get_ref_crick(n_P, n_phi)[:len(n_crick_mut)]
        c_crick_ref = get_ref_crick(c_P, c_phi)[:len(c_crick_mut)]

        n_crick_diff = diffangle(n_crick_mut, n_crick_ref)
        n_crick_diff = (n_crick_diff[0::2] + n_crick_diff[1::2])/2

        c_crick_diff = diffangle(c_crick_mut, c_crick_ref)
        c_crick_diff = (c_crick_diff[0::2] + c_crick_diff[1::2])/2

        n_crick_diff = np.mean(n_crick_diff)
        c_crick_diff = np.mean(c_crick_diff)

        # rotation asymmetry
        crick_diff = diffangle(n_crick_diff, c_crick_diff) 

        seq1 = bundle_df.res_name[0::4].tolist()
        seq2 = bundle_df.res_name[1::4].tolist()
        seq1 = "".join([AA_3_to_1[res] for res in seq1])
        seq2 = "".join([AA_3_to_1[res] for res in seq2])
        
        assert seq1 == h1 and seq2 == h2

        # add record
        
        print(h1, h2, idx, true_id, group, pdb_file_simple, scores['ptm'])
        
        data.append(
                (true_id, group, n_crick_diff, c_crick_diff, crick_diff, \
                 h1, h2, n_crick_mut, c_crick_mut, seq, pdb_file_simple,
                 scores['max_pae'], np.mean(scores['plddt']), scores['ptm'], np.mean(scores['pae']))
            )
        
    # debug
    #break
        
data_df = pd.DataFrame(data, columns=['idx', 'group', 'n_rot', 'c_rot', 'rot', 
                                      'n_seq', 'c_seq', 'n_crick_mut', 'c_crick_mut', 'full_seq', 'pdb_file',
                                      'max_pae', 'plddt', 'ptm', 'pae_mean'])
assert data_df['idx'].is_unique
data_df.set_index('idx', inplace=True)
len(data_df)

PLKELVQGVQRIA EIGELITSFNLMA 0 0 A_group_13 0/hamp_0_relaxed_rank_1_model_4.pdb 0.85
PIRELVEGVRSIA ELGELIASFNDMA 1 1 A_group_13 1/hamp_1_relaxed_rank_1_model_2.pdb 0.85
PIKELLDGVKNIA ELGELIVNFNEMA 2 2 A_group_13 2/hamp_2_relaxed_rank_1_model_2.pdb 0.84
PLSELRKGIRRVA ELGDLILQFNDMG 14 14 A_group_13 14/hamp_14_relaxed_rank_1_model_2.pdb 0.83
PVKELLRGVRAVA ELGELVNGFNDMA 15 15 A_group_13 15/hamp_15_relaxed_rank_1_model_1.pdb 0.81
PIKELLAGIRNIS QIGELIFSFNNMA 16 16 A_group_13 16/hamp_16_relaxed_rank_1_model_4.pdb 0.89
PQKKLLLGIQNIA QLSTLIISFNEMA 17 17 A_group_13 17/hamp_17_relaxed_rank_1_model_4.pdb 0.79
PLDILSNGVMQIS EFAPVCADFNEMA 18 18 A_group_14 18/hamp_18_relaxed_rank_1_model_2.pdb 0.85
PLELLSYGAEQIK EFGQVCGDFDEMR 19 19 A_group_14 19/hamp_19_relaxed_rank_1_model_4.pdb 0.82
PLELLSYGAGQIE EFAKVCRDFDKMR 20 20 A_group_14 20/hamp_20_relaxed_rank_1_model_4.pdb 0.82
PLQKLSRGTKEIK EFGEVCRDFDEMR 21 21 A_group_14 21/hamp_21_relaxed_rank_1_model_2.pdb 0.85
PLVKLKQAAEKMG EVGELCESFENMR 22 22 A_group_14

PVSNTARVSQRLS EVARLSRSFNRMA 134 134 A_group_18 134/hamp_134_relaxed_rank_1_model_4.pdb 0.84
PIRVAAHTSERLA VLASLARSFNGMA 135 135 A_group_18 135/hamp_135_relaxed_rank_1_model_4.pdb 0.88
PVRQAAQIAERLA DVSRLAESFNEMA 136 136 A_group_18 136/hamp_136_relaxed_rank_1_model_3.pdb 0.8
PVRTGAEVARLIA EIAVLGESFNDMA 137 137 A_group_18 137/hamp_137_relaxed_rank_1_model_4.pdb 0.83
PVRQAASSAEKLS DLARLAVSFNDMA 139 139 A_group_18 139/hamp_139_relaxed_rank_1_model_4.pdb 0.84
PVSQAALVAEKLA EVARLAESFNKMA 140 140 A_group_18 140/hamp_140_relaxed_rank_1_model_4.pdb 0.78
PVRVAAQIAERLA DLARLAASFNKMA 141 141 A_group_18 141/hamp_141_relaxed_rank_1_model_4.pdb 0.77
PVRAASKIAEKLA EVARLAFSFNDMA 142 142 A_group_18 142/hamp_142_relaxed_rank_1_model_4.pdb 0.85
PVRRAAEVAERLA DLALLGTSFNAMA 143 143 A_group_18 143/hamp_143_relaxed_rank_1_model_4.pdb 0.82
PVQQAARAAESLS EMARLSMSFNRMA 144 144 A_group_18 144/hamp_144_relaxed_rank_1_model_2.pdb 0.86
PIRLARKVAERLA DLARLATSFNQMA 146 146 A_group_18 146/hamp_146_relaxed_rank_1_model_

PMTRMVGTMRALK ELGQLARQFDQLL 251 251 A_group_38 251/hamp_251_relaxed_rank_1_model_4.pdb 0.82
PLEQMNRTIAAVD EIGLVARRFDELL 254 254 A_group_38 254/hamp_254_relaxed_rank_1_model_4.pdb 0.8
PIERMHATIGRID ELGRLATAFDHLL 255 255 A_group_38 255/hamp_255_relaxed_rank_1_model_4.pdb 0.84
PIERMHATMHAIE ELGVVAAHFDRLL 260 260 A_group_38 260/hamp_260_relaxed_rank_1_model_2.pdb 0.76
PIERMAAVMGRIE ELGGLARSFDRLL 261 261 A_group_38 261/hamp_261_relaxed_rank_1_model_2.pdb 0.74
PIERMHGTIGRIE ELGLLAREFDRLL 264 264 A_group_38 264/hamp_264_relaxed_rank_1_model_4.pdb 0.82
PIQHLINTSKEIA EIGQLAITFNEMA 265 265 A_group_44 265/hamp_265_relaxed_rank_1_model_2.pdb 0.83
PVRKLGRATRELA ELADLAKTFNETA 266 266 A_group_44 266/hamp_266_relaxed_rank_1_model_2.pdb 0.75
PLNDIISVSKEIT ELGQLAVHFNKMA 267 267 A_group_44 267/hamp_267_relaxed_rank_1_model_1.pdb 0.82
PVRALNTAASQLG ELAQLVTTFNHTA 268 268 A_group_44 268/hamp_268_relaxed_rank_1_model_4.pdb 0.88
SLQKLQAGTAEIA EVEDLAKSFNYMS 269 269 A_group_44 269/hamp_269_relaxed_rank_1_model_

PLRRLRTGALDVA EIGEVARTFDELQ 376 382 A_group_506 376/hamp_376_relaxed_rank_1_model_3.pdb 0.78
SLRDLRQGALLIA EFGQVTEAFNAVH 378 384 A_group_506 378/hamp_378_relaxed_rank_1_model_1.pdb 0.85
AHERLPTALRQIR EVGQLARAVDVLH 379 385 A_group_506 379/hamp_379_relaxed_rank_1_model_2.pdb 0.36
SEQQLPELVARMR EIGQVGRAFTAVR 380 386 A_group_506 380/hamp_380_relaxed_rank_1_model_2.pdb 0.27
RLKRLRGETLALA EIGQVARAFEHAH 381 387 A_group_506 381/hamp_381_relaxed_rank_1_model_3.pdb 0.79
ARRKLPEAMRRLR EAGQVAEALDTVH 382 388 A_group_506 382/hamp_382_relaxed_rank_1_model_4.pdb 0.38
SLRLLREGALAVA EVGQVALAFNVVH 383 402 A_group_506 383/hamp_383_relaxed_rank_1_model_3.pdb 0.84
PLERLARAADTLG EVARASKAFNAMQ 384 363 A_group_509 384/hamp_384_relaxed_rank_1_model_2.pdb 0.6
PLRRLANAAEALG PDDVTAIAFNRMQ 385 364 A_group_509 385/hamp_385_relaxed_rank_1_model_2.pdb 0.79
PLKELSVAAEKFG EVRSLTSAFNDMQ 386 365 A_group_509 386/hamp_386_relaxed_rank_1_model_4.pdb 0.86
PIKLFASAAERLG EIEVAARAFNEMQ 387 366 A_group_509 387/hamp_387_relaxed_ra

PIRLLISAADSVA DVGRLSRTFNKMV 482 495 A_group_63 482/hamp_482_relaxed_rank_1_model_4.pdb 0.91
PVSGLVQAAGRVS EIRALSIAFNMMT 483 496 A_group_63 483/hamp_483_relaxed_rank_1_model_2.pdb 0.92
PIRRLINAADQVA DLQHLGESFNKMT 484 497 A_group_63 484/hamp_484_relaxed_rank_1_model_4.pdb 0.9
PLGLLILAAQRIS EVTGLSRAFNSMT 486 499 A_group_63 486/hamp_486_relaxed_rank_1_model_4.pdb 0.77
PISRLVAAAERVR EIAALARAFNRMT 491 504 A_group_63 491/hamp_491_relaxed_rank_1_model_2.pdb 0.88
PIGRLIDAAEKVR EIGVLSRAFNRMT 492 505 A_group_63 492/hamp_492_relaxed_rank_1_model_2.pdb 0.89
PVGRLAGAVQRVG EIAMLGRGFNQMT 493 506 A_group_63 493/hamp_493_relaxed_rank_1_model_2.pdb 0.85
PLGTLIGATRTAA EISVLTRAFNRMT 494 507 A_group_63 494/hamp_494_relaxed_rank_1_model_4.pdb 0.91
PIGRLAEASEQVG EIQTLGESFNRMT 500 513 A_group_63 500/hamp_500_relaxed_rank_1_model_2.pdb 0.86
PLGLLILASGRIS ELSSLSRAFNRMT 501 514 A_group_63 501/hamp_501_relaxed_rank_1_model_2.pdb 0.89
PIRRLMSAADHVA DLASLAETFNKMT 509 522 A_group_63 509/hamp_509_relaxed_rank_1_model_

...

PIKLCAERIEKLS EVKILAKSAAKVV 6114 6110 BACKGROUND 6114/hamp_6114_relaxed_rank_1_model_3.pdb 0.78
PLTDISMFADEVA EFGNVAKALNNTA 6115 6111 BACKGROUND 6115/hamp_6115_relaxed_rank_1_model_4.pdb 0.77
SIKEIAAQTERVA EFGSLADSIEQMR 6116 6112 BACKGROUND 6116/hamp_6116_relaxed_rank_1_model_2.pdb 0.89
TVAEYLAFTRRIA ALGQLGEGLNAMV 6118 6114 BACKGROUND 6118/hamp_6118_relaxed_rank_1_model_4.pdb 0.87
PISQLTDAAKKLA EVGTLAEMFREMS 6119 6115 BACKGROUND 6119/hamp_6119_relaxed_rank_1_model_4.pdb 0.8
PLSAAVHALRSVA EFGQLGVALNDTV 6120 6116 BACKGROUND 6120/hamp_6120_relaxed_rank_1_model_2.pdb 0.84
PLKKISSTAKEVA EIGEVASSLNLVV 6121 6117 BACKGROUND 6121/hamp_6121_relaxed_rank_1_model_2.pdb 0.86
PLTKLTHKAQQVA EVKTLANNFNDLI 6122 6118 BACKGROUND 6122/hamp_6122_relaxed_rank_1_model_2.pdb 0.73
PLSYLVDVARNIA EIKTIEKSFEKMS 6123 6119 BACKGROUND 6123/hamp_6123_relaxed_rank_1_model_4.pdb 0.61
RVDQMVKAVKGGA ELGRLGDDFNTML 6124 6120 BACKGROUND 6124/hamp_6124_relaxed_rank_1_model_4.pdb 0.82
PLQHLTNVTLEIA EIGQLCDNFNQMT 6126 6122 BAC

PLRRVAQFASELT EIDLVVEGFSVLQ 6211 6207 BACKGROUND 6211/hamp_6211_relaxed_rank_1_model_4.pdb 0.87
RLTLGAQAARKIA EVAAFASAVDTAV 6212 6208 BACKGROUND 6212/hamp_6212_relaxed_rank_1_model_4.pdb 0.88
IIDQINQVMRKVN RLTELKTFINQTV 6213 6209 BACKGROUND 6213/hamp_6213_relaxed_rank_1_model_1.pdb 0.84
NLKRLKRGTQRLG ELGDLAIAFNTMS 6214 6210 BACKGROUND 6214/hamp_6214_relaxed_rank_1_model_3.pdb 0.82
PVRRVAEAALRLA ELHELARTFDQMA 6215 6211 BACKGROUND 6215/hamp_6215_relaxed_rank_1_model_1.pdb 0.66
PMRTLRLALDRIS EVGRLEHSVNELA 6216 6212 BACKGROUND 6216/hamp_6216_relaxed_rank_1_model_2.pdb 0.89
PLAELSEASKRIQ EIGELVSSFENMR 6217 6213 BACKGROUND 6217/hamp_6217_relaxed_rank_1_model_5.pdb 0.89
PVLSLEKTVNELD ETKALARGINRLA 6218 6214 BACKGROUND 6218/hamp_6218_relaxed_rank_1_model_1.pdb 0.85
TIGTICAGMEIVK RFEEFRRDFNTLI 6219 6215 BACKGROUND 6219/hamp_6219_relaxed_rank_1_model_1.pdb 0.56
PIVRLRSDIESID EYLALIHEINRLL 6220 6216 BACKGROUND 6220/hamp_6220_relaxed_rank_1_model_3.pdb 0.82
NLVRKMHDTGSDA ELGRLEQRFERMR 6221 6217 BA

PLRRLADAAHQLA EVADVGAALDGLA 6310 6310 BACKGROUND 6310/hamp_6310_relaxed_rank_1_model_1.pdb 0.82
PLEDLSRHCRAVT EIDQVARTHNEML 6311 6311 BACKGROUND 6311/hamp_6311_relaxed_rank_1_model_4.pdb 0.85
SIESATNVATELS RSDKLGHAMNSLA 6312 6312 BACKGROUND 6312/hamp_6312_relaxed_rank_1_model_2.pdb 0.88
NIDIITDGLSTLA EMGQISQSVNNLA 6313 6313 BACKGROUND 6313/hamp_6313_relaxed_rank_1_model_2.pdb 0.83
PLEWAARKAEAMS EVAALVKALNGML 6314 6314 BACKGROUND 6314/hamp_6314_relaxed_rank_1_model_2.pdb 0.75
PIEALAKEVRELE ELTSLVRNLNRLL 6315 6315 BACKGROUND 6315/hamp_6315_relaxed_rank_1_model_2.pdb 0.83
HLNKISQYAHTLT ELDTVVTALNEMQ 6316 6316 BACKGROUND 6316/hamp_6316_relaxed_rank_1_model_1.pdb 0.75
PLQQLSEQMQNQA ELQVLIDSFGAMR 6318 6318 BACKGROUND 6318/hamp_6318_relaxed_rank_1_model_4.pdb 0.41
PIKYLYSTIEKQK EIGVTIKEFFNLQ 6319 6319 BACKGROUND 6319/hamp_6319_relaxed_rank_1_model_2.pdb 0.77
YLNRIMKDTKDMA PLASHAENLNHLR 6320 6320 BACKGROUND 6320/hamp_6320_relaxed_rank_1_model_2.pdb 0.88
PLKALTRRVGDFA DIQGLIDAHNAME 6321 6321 BA

KIRKISVSAEAIV DIGYLIYQINCMS 6401 6401 BACKGROUND 6401/hamp_6401_relaxed_rank_1_model_5.pdb 0.42
DLKRLRAAANAFG HTRDIHSAFNAMA 6402 6402 BACKGROUND 6402/hamp_6402_relaxed_rank_1_model_5.pdb 0.88
PVKQLVEQLSKVA ELNAVKDSVNQLL 6403 6403 BACKGROUND 6403/hamp_6403_relaxed_rank_1_model_4.pdb 0.7
PLHSLLEEISTLD EVEELTNAFNALL 6404 6404 BACKGROUND 6404/hamp_6404_relaxed_rank_1_model_2.pdb 0.8
AIARLSEATARTG ELLPLGEALNTMR 6405 6405 BACKGROUND 6405/hamp_6405_relaxed_rank_1_model_2.pdb 0.88
PVRELATQMQALK EVHFLGATFNDMV 6406 6406 BACKGROUND 6406/hamp_6406_relaxed_rank_1_model_4.pdb 0.76
QIAGLAQQVKTLR EVAELVDAFNDYH 6407 6407 BACKGROUND 6407/hamp_6407_relaxed_rank_1_model_1.pdb 0.74
HVNRLLESMASLQ ELRQIQQGIRALG 6408 6408 BACKGROUND 6408/hamp_6408_relaxed_rank_1_model_2.pdb 0.89
PIVTMSNKMNQIR ETDNLIDTFNEMM 6409 6409 BACKGROUND 6409/hamp_6409_relaxed_rank_1_model_4.pdb 0.86
PLSQLKTELKKVE EIKEVEQSVYEMA 6410 6410 BACKGROUND 6410/hamp_6410_relaxed_rank_1_model_4.pdb 0.89
TIRRVSLRISEFS ELDTLASRFERLM 6411 6411 BACK

5314

In [32]:
data_df.to_pickle(os.path.join(data_dir, 'af2.p'))

In [33]:
# checked, OK